In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import numpy as np
import pandas as pd
import glob
import os.path
import datetime
import os
import psycopg2
import csv
from datetime import datetime

In [10]:
def read_plt(plt_file):
    points = pd.read_csv(plt_file, skiprows=6, header=None)
    points['time'] = pd.to_datetime(points[5].astype(str) + ' ' + points[6].astype(str))
    points.rename(inplace=True, columns={0: 'lat', 1: 'lon', 3: 'alt'})
    points.drop(inplace=True, columns=[2, 4, 5, 6])
    return points

mode_names = ['walk', 'bike', 'bus', 'car', 'subway','train', 'airplane', 'boat', 'run', 'motorcycle', 'taxi']
mode_ids = {s : i + 1 for i, s in enumerate(mode_names)}

def read_labels(labels_file):
    labels = pd.read_csv(labels_file, skiprows=1, header=None, delim_whitespace=True)
    labels['start_time'] = pd.to_datetime(labels[0].astype(str) + ' ' + labels[1].astype(str))
    labels['end_time'] = pd.to_datetime(labels[2].astype(str) + ' ' + labels[3].astype(str))
    labels = labels.drop(columns=[0, 1, 2, 3])  # Drop the original columns after conversion
    labels['label'] = [mode_ids[i] for i in labels[4]]
    labels = labels.drop(columns=[4])  # Drop the 'label' column after it's processed
    return labels


def apply_labels(points, labels):
    indices = labels['start_time'].searchsorted(points['time'], side='right') - 1
    no_label = (indices < 0) | (points['time'].values >= labels['end_time'].iloc[indices].values)
    points['label'] = labels['label'].iloc[indices].values
    points['label'][no_label] = 0

def read_user(user_folder):
    labels = None
    plt_files = glob.glob(os.path.join(user_folder, 'Trajectory', '*.plt'))
    df = pd.concat([read_plt(f) for f in plt_files])
    labels_file = os.path.join(user_folder, 'labels.txt')
    if os.path.exists(labels_file):
        labels = read_labels(labels_file)
        apply_labels(df, labels)
    else:
        df['label'] = 0
    return df

def read_all_users(folder):
    subfolders = os.listdir(folder)
    dfs = []
    for i, sf in enumerate(subfolders):
        if sf == '.DS_Store':
            continue
        print('[%d/%d] processing user %s' % (i + 1, len(subfolders), sf))
        df = read_user(f"data/{sf}")
        df['user_id'] = int(sf)
        dfs.append(df)
    return pd.concat(dfs)

In [11]:
df = read_all_users(folder="data")

[2/11] processing user 021
[3/11] processing user 056
[4/11] processing user 058
[5/11] processing user 064
[6/11] processing user 069
[7/11] processing user 075
[8/11] processing user 080
[9/11] processing user 154
[10/11] processing user 170
[11/11] processing user 175


In [12]:
df.head()

,lat,lon,alt,time,label,user_id
0,39.975567,116.330350,226.377953,2007-04-29 08:34:32,0,21
1,39.975450,116.330233,301.837270,2007-04-29 08:37:00,0,21
2,39.973833,116.332967,328.083990,2007-04-29 12:34:24,11,21
3,39.973900,116.332733,328.083990,2007-04-29 12:34:32,11,21
4,39.973800,116.332867,328.083990,2007-04-29 12:34:46,11,21


In [13]:
df.to_pickle('loaded/geolife.pkl')
df.to_csv('loaded/trajectories.csv', index=False)

In [14]:
# Database connection details
conn = psycopg2.connect(
    host="localhost",      # Hostname
    port="25432",          # Port
    dbname="mobilitydb",   # Database name
    user="skandar",        # Username
    password="skandar"     # Password
)

# Create a cursor
cursor = conn.cursor()

# Create the table if it doesn't already exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS trajectories (
        id SERIAL PRIMARY KEY,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        altitude DOUBLE PRECISION,
        timestamp TIMESTAMP,
        label INT,
        user_id INT,
        geom GEOMETRY(Point, 4326)
    );
""")

# Open the CSV file and read the data
with open('loaded/trajectories.csv', 'r') as file:
    csvreader = csv.reader(file)
    
    # Skip the header line
    next(csvreader)
    
    # Loop over each row in the CSV file
    for row in csvreader:
        # Extract values from the CSV row
        lat = float(row[0])  # Latitude
        lon = float(row[1])  # Longitude
        alt = float(row[2])  # Altitude
        timestamp = datetime.strptime(row[3], '%Y-%m-%d %H:%M:%S')  # Timestamp
        label = int(row[4])  # Label
        user_id = int(row[5])   # User
        
        # Insert data into the database
        cursor.execute("""
            INSERT INTO trajectories (latitude, longitude, altitude, timestamp, label, user_id, geom)
            VALUES (%s, %s, %s, %s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326));
        """, (lat, lon, alt, timestamp, label, user_id, lon, lat))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Data successfully loaded into the database.")

Data successfully loaded into the database.
